# 04 — Spectra: Lines, Noise, and Redshift

## What you’ll learn
- Building a 1D spectrum model
- Estimating noise and SNR
- Continuum normalization
- Fitting spectral lines (Gaussian)
- Estimating redshift from line centers
- Equivalent width (EW)

We’ll use a synthetic spectrum so the notebook runs anywhere.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

np.random.seed(3)


## 1) Create a synthetic spectrum

We'll simulate:
- continuum (slowly varying)
- absorption lines (Gaussians dipping below continuum)
- additive Gaussian noise


In [ ]:
lam = np.linspace(4000, 7000, 4000)  # Angstrom
continuum = 1.0 + 0.05*np.sin(2*np.pi*(lam-4000)/3000)

def gauss(x, mu, sig):
    return np.exp(-0.5*((x-mu)/sig)**2)

# rest-frame absorption lines
lines_rest = [
    (4861.0, 1.8, 0.25),  # H-beta like (mu, sigma, depth)
    (6563.0, 2.2, 0.30),  # H-alpha like
]

z_true = 0.023
flux = continuum.copy()

for mu, sig, depth in lines_rest:
    flux *= (1.0 - depth*gauss(lam, mu*(1+z_true), sig))

noise_sigma = 0.01
flux_noisy = flux + np.random.normal(0, noise_sigma, size=lam.size)

plt.figure(figsize=(9,4))
plt.plot(lam, flux_noisy, linewidth=1)
plt.xlabel("wavelength [Å]")
plt.ylabel("flux (arb.)")
plt.title("Synthetic observed spectrum (with noise)")
plt.tight_layout()
plt.show()


## 2) Continuum normalization

A common workflow:
1. estimate continuum (smooth or fit)
2. divide spectrum by continuum → baseline ~ 1

Here we’ll use a Gaussian smoothing as a simple continuum estimate.


In [ ]:
from scipy.ndimage import gaussian_filter1d

cont_est = gaussian_filter1d(flux_noisy, sigma=80)
norm = flux_noisy / cont_est

plt.figure(figsize=(9,4))
plt.plot(lam, norm, linewidth=1, label="normalized")
plt.axhline(1.0, linestyle="--")
plt.xlabel("wavelength [Å]")
plt.ylabel("normalized flux")
plt.title("Continuum-normalized spectrum")
plt.tight_layout()
plt.show()


## 3) Fit a Gaussian absorption line

Model (normalized):
\[
f(\lambda) = 1 - A \exp\left(-\frac{(\lambda-\mu)^2}{2\sigma^2}\right)
\]

We’ll fit around an expected line region.


In [ ]:
def absorption_model(x, A, mu, sig):
    return 1.0 - A*np.exp(-0.5*((x-mu)/sig)**2)

# pick the H-alpha-like region
win = (lam > 6500) & (lam < 6620)
x = lam[win]
y = norm[win]

p0 = [0.2, 6563*(1+0.02), 2.0]
popt, pcov = curve_fit(absorption_model, x, y, p0=p0)
A_hat, mu_hat, sig_hat = popt

print("Fit params:")
print(" A  =", A_hat)
print(" mu =", mu_hat)
print(" sig=", sig_hat)

plt.figure(figsize=(8,4))
plt.plot(x, y, ".", markersize=3, label="data")
plt.plot(x, absorption_model(x, *popt), "-", linewidth=2, label="fit")
plt.xlabel("wavelength [Å]")
plt.ylabel("normalized flux")
plt.title("Gaussian absorption-line fit")
plt.legend()
plt.tight_layout()
plt.show()


## 4) Estimate redshift

If you know the rest wavelength:
\[
z = \frac{\mu_{obs}}{\mu_{rest}} - 1
\]


In [ ]:
mu_rest = 6563.0
z_hat = mu_hat/mu_rest - 1
print("True z:", z_true)
print("Estimated z:", z_hat)


## 5) Equivalent width (EW)

For a normalized absorption line:
\[
EW = \int (1 - f_{norm})\, d\lambda
\]

We’ll integrate in the fitting window.


In [ ]:
ew = np.trapz(1 - absorption_model(x, *popt), x)
print("EW estimate [Å]:", ew)


## Try it
- Change the smoothing scale for continuum estimation and see how it affects EW.
- Increase the noise and see the uncertainty in mu (and z).
- Add a second nearby line and see how single-Gaussian fits can fail.

Next notebook: **05 — Catalogs, Coordinates & Crossmatch**
